# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [133]:
import pandas as pd 
import numpy as np 

# Plotting 
import seaborn as sns
import matplotlib.pyplot as plt

# Stats stuff
import scipy.stats as stats

In [134]:
#Your code here
df = pd.read_csv("homepage_actions.csv")
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [135]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [136]:
df['id'].duplicated().value_counts()

False    6328
True     1860
Name: id, dtype: int64

In [137]:
df.groupby('group')['action'].value_counts()

group       action
control     view      3332
            click      932
experiment  view      2996
            click      928
Name: action, dtype: int64

In [138]:
df_control = df[df['group'] == 'control'].reset_index(drop=True)
print(df_control.shape)
df_control.head()

(4264, 4)


,timestamp,id,group,action
0,2016-09-24 19:59:02.646620,671993,control,view
1,2016-09-24 21:00:12.278374,560027,control,view
2,2016-09-25 00:25:14.141290,281985,control,view
3,2016-09-25 01:14:48.603202,407864,control,view
4,2016-09-25 02:16:11.046654,342984,control,view


In [139]:
df_experiment = df[df['group'] == 'experiment'].reset_index(drop=True)
print(df_experiment.shape)
df_experiment.head()

(3924, 4)


,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 20:26:14.466886,536734,experiment,view
4,2016-09-24 20:32:25.712659,681598,experiment,view


In [140]:
#check if there is overlap between the control and experiment groups
df_experiment['id'].isin(df_control['id']).value_counts()

False    3924
Name: id, dtype: int64

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [141]:
#Your code here
ctrl_views_count = sum(df_control['action'] == 'view')
expr_views_count = sum(df_experiment['action'] == 'view')

print(f"ctrl_views_count: {ctrl_views_count}")
print(f"expr_views_count: {expr_views_count}")

ctrl_views_count: 3332
expr_views_count: 2996


In [142]:
ctrl_clicks_count = sum(df_control['action'] == 'click')
expr_clicks_count = sum(df_experiment['action'] == 'click')

print(f"ctrl_clicks_count: {ctrl_clicks_count}")
print(f"expr_clicks_count: {expr_clicks_count}")

ctrl_clicks_count: 932
expr_clicks_count: 928


In [143]:
# "converted" vs "not converted"
observations = np.array([expr_clicks_count, expr_views_count - expr_clicks_count])

expectations = np.array([ctrl_clicks_count, ctrl_views_count - ctrl_clicks_count])

print('OBSERVED (experiment):', observations) 
print('EXPECTED (control):', expectations)

OBSERVED (experiment): [ 928 2068]
EXPECTED (control): [ 932 2400]


In [144]:
chi2, p_value = stats.chisquare(f_obs=observations, f_exp=expectations)
print(f"chi2: {chi2}")
print(f"p_value: {p_value}")

chi2: 45.943834048640916
p_value: 1.2169234221091101e-11


In [145]:
# We'll use our observations (experiment group) & expecations (control group) as
# defined earlier in the lecture
contingency_table = np.array([observations, expectations])
contingency_table

array([[ 928, 2068],
       [ 932, 2400]])

In [146]:
stats.chi2_contingency(contingency_table)

(6.712921132285344,
 0.00957168049704227,
 1,
 array([[ 880.61946903, 2115.38053097],
        [ 979.38053097, 2352.61946903]]))

In [147]:
experiment_percent = expr_clicks_count/expr_views_count*100
print(f'Percent Experiment Converted: {experiment_percent:.5}%')
control_percent = ctrl_clicks_count/ctrl_views_count*100
print(f'Percent Control Converted: {control_percent:.5}%')

print(f'Difference between experiment & control {experiment_percent-control_percent:.3}%')

Percent Experiment Converted: 30.975%
Percent Control Converted: 27.971%
Difference between experiment & control 3.0%


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [148]:
#Your code here
control_percent = ctrl_clicks_count/ctrl_views_count*100
control_percent

27.971188475390157

In [149]:
# expected number of clicks for the experiment group, 
#if it had the same click-through rate as that of the control group.
experiment_percent = 27.971188475390157 
expr_clicks_count_expected = experiment_percent*expr_views_count/100 
expr_clicks_count_expected

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [150]:
# Calculate standard deviation for clicks
click_rate = control_percent/100
experiment_group_size = len(df_experiment)
std_dev = np.sqrt( experiment_group_size * click_rate * (1 - click_rate))
std_dev

28.117265621107368

In [151]:
# number of standard deviations from the actual number of clicks 
num_std = (expr_clicks_count - expr_clicks_count_expected)/std_dev
num_std

3.200282505769741

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [152]:
#Your code here
# z_score is the number of standard deviations from the estimate value seen above 
z_score = num_std
p_value = stats.norm.sf(z_score) * 2
print(f"p_values: {p_value}") 

p_values: 0.00137292944719451


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **My analysis here**
> 
> Yes this matches the previous statistical test. The p_values are very small and less than alpha(0.05) so that at 95% significance level we reject the null hypothesis which states that there is no difference between the control and experimental groups and support the alternative one which states that the  experimental homepage was more effective than that of the control group.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.